In [1]:
from selenium import webdriver   
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import os
import pandas as pd
import time
from datetime import date
import string

Choose the expiration time by using
Dec 2016, Jan 2017 and Feb 2017.

In [2]:
stocks = ['GOOG','YHOO','AAPL','MSFT','AMZN','JPM','BAC','HSBC','CIT','GS']

In [3]:
#12.16; 1.20, 2.17; 
dates = ['1481846400','1484870400','1487289600']
maturity_dates = [date(2016,12,16),date(2016,1,20),date(2016,2,17)]

In [19]:
(date(2016,12,16) - date.today()).days/365

0.18356164383561643

** Example **

In [21]:
url = 'http://finance.yahoo.com/quote/GOOG/options?straddle=true&date=1481846400'
chromedriver = "/Users/Miya/Downloads/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(url)
html_source = driver.page_source 
table = soup.findAll('table','straddles')
table_html = str(table)
HTML(table_html)

Last Price,Change,% Change,Volume,Open Interest,Strike,Last Price,Change,% Change,Volume,Open Interest
302.80,0.00,0.00%,1,4,440.00,0.20,0.00,0.00%,2,6
323.00,30.10,10.28%,1,4,450.00,0.10,0.00,0.00%,1,77
261.00,0.00,0.00%,4,6,460.00,0.65,0.00,0.00%,1,1
251.10,0.00,0.00%,2,7,470.00,1.00,0.00,0.00%,1,23
292.80,0.00,0.00%,1,35,480.00,0.31,0.00,0.00%,7,115
-,-,-,-,-,490.00,1.05,0.00,0.00%,1,22
280.10,0.00,0.00%,1,9,500.00,0.20,0.00,0.00%,25,106
270.90,0.00,0.00%,1,4,520.00,0.25,-0.25,-50.00%,1,62
248.77,0.00,0.00%,1,9,540.00,0.59,0.00,0.00%,50,120
211.70,0.00,0.00%,1,46,560.00,0.50,-0.20,-28.57%,1,81


In [12]:
pwd

"C:\\Users\\Miya\\OneDrive\\Miya'sGithub\\Financial-Modeling"

In [117]:
mkdir data-quiz5

In [13]:
cd data-quiz5

C:\Users\Miya\OneDrive\Miya'sGithub\Financial-Modeling\data-quiz5


## Crawl All Data

**Option Data on Yahoo Finance is dynamic.(Javascript) Therefore, small tricks are needed when BeautifulSoup is used.**

In [4]:
chromedriver = "/Users/Miya/Downloads/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
df_dict = {}
today = date.today()
for stock in stocks:
    for date,maturity in zip(dates,maturity_dates):
        ## Get the url
        url = 'http://finance.yahoo.com/quote/'+ stock +'/options?straddle=true&date='+ date
        ## Crawl data
        driver.get(url)
        html_source = driver.page_source 
        ## Beautifulsoup
        soup = BeautifulSoup(html_source,'html.parser')  
        
        if soup.find('table','straddles') is not None:
            
            title = [i.text for i in soup.find('table','straddles').find_all('th')]
            text = [i.text for i in soup.find('table','straddles').find_all('td')]
            rows = [row for row in soup.find('table','straddles').find_all("tr")]

            l_table = len(rows) - 1
        ## call data 
            dictionary_call = {}
            dictionary_call['type'] =  ['Call']*l_table
            dictionary_call['option'] = [stock]*l_table
            dictionary_call['maturity_date'] = [maturity]*l_table
            dictionary_call['date'] = [today]*l_table
        
            for j in range(6):
                key = title[j]
                dictionary_call[key] = []
                for i in range(l_table):
                    dictionary_call[key].append(text[11*i+j])
                
                
        ## put data        
            dictionary_put = {}
            dictionary_put['type'] =  ['Put']*l_table
            dictionary_put['option'] = [stock]*l_table
            dictionary_put['maturity_date'] = [maturity]*l_table
            dictionary_put['date'] = [today]*l_table
        
            for j in range(6,11):
                key = title[j]
                dictionary_put[key] = []
                for i in range(l_table):
                    dictionary_put[key].append(text[11*i+j])
                
        ## write into dataframe
            df_put = pd.DataFrame(dictionary_put)
            df_call = pd.DataFrame(dictionary_call)
            
            df = pd.concat([df_call,df_put],ignore_index =  True)        
            #df.to_csv(stock+date+'.csv')
            
            df_name = stock + date
            df_dict[df_name] = df

In [5]:
print('We now have %d dataframes' % len(df_dict.keys()))

We now have 23 dataframes


** Sample DataFrame **

In [24]:
df_dict['AAPL1481846400'].columns

Index(['% Change', 'Change', 'Last Price', 'Open Interest', 'Strike', 'Volume',
       'date', 'maturity_date', 'option', 'type'],
      dtype='object')

In [29]:
df_dict['AAPL1481846400'].head()

,% Change,Change,Last Price,Open Interest,Strike,Volume,date,maturity_date,option,type
0,0.00%,0.00,86.20,0,30.00,66,2016-10-10,2016-12-16,AAPL,Call
1,0.00%,0.00,71.55,0,45.00,7,2016-10-10,2016-12-16,AAPL,Call
2,0.00%,0.00,66.35,0,50.00,14,2016-10-10,2016-12-16,AAPL,Call
3,-,-,-,-,55.00,-,2016-10-10,2016-12-16,AAPL,Call
4,3.82%,2.07,56.27,10,60.00,5,2016-10-10,2016-12-16,AAPL,Call


## Implied Volatility Surface

In [6]:
import numpy as np
import matplotlib.pyplot as plot
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

**Implied volatility surface need time to maturity and strike price**

In [7]:
pwd

"C:\\Users\\Miya\\OneDrive\\Miya'sGithub\\Financial-Modeling"

In [8]:
cd ../

C:\Users\Miya\OneDrive\Miya'sGithub


In [33]:
mkdir implied-volatility-surface

In [9]:
cd implied-volatility-surface

[WinError 2] The system cannot find the file specified: 'implied-volatility-surface'
C:\Users\Miya\OneDrive\Miya'sGithub


### Prepare dataframe

In [75]:
df_dict['AAPL1481846400'].dtypes

% Change          object
Change            object
Last Price        object
Open Interest     object
Strike           float64
Volume            object
date              object
maturity_date     object
option            object
type              object
dtype: object

In [10]:
## remove , in dataframe
def remove_punctuation(dataframe):
    columns_to_set = ['Last Price','Open Interest','Strike','Volume']
    for i in columns_to_set:
        series = dataframe[i]
        series_new = []
        for j in series:
            j = str(j)
            j_new = ''.join(ch for ch in j if ch != ',')
            series_new.append(j_new)
        dataframe[i] = series_new
    return dataframe

In [11]:
for key in df_dict.keys():
    dataframe = df_dict[key]
    df_dict[key] = remove_punctuation(dataframe)

In [12]:
df_dict['AAPL1481846400'].head()

,% Change,Change,Last Price,Open Interest,Strike,Volume,date,maturity_date,option,type
0,0.00%,0.00,86.20,0,30.00,66,2016-10-11,2016-12-16,AAPL,Call
1,0.00%,0.00,71.55,0,45.00,7,2016-10-11,2016-12-16,AAPL,Call
2,0.00%,0.00,66.35,0,50.00,14,2016-10-11,2016-12-16,AAPL,Call
3,-,-,-,-,55.00,-,2016-10-11,2016-12-16,AAPL,Call
4,3.82%,2.07,56.27,10,60.00,5,2016-10-11,2016-12-16,AAPL,Call


In [15]:
## change the dtype 
columns_to_change = ['Last Price','Open Interest','Strike','Volume']
for key in df_dict.keys():
    ## detect nan
    df_dict[key] = df_dict[key].replace('-',np.nan)
    for i in columns_to_change:
        df_dict[key][i] = df_dict[key][i].astype(float)

In [16]:
## check the dtype
df_dict['AAPL1481846400'].dtypes

% Change          object
Change            object
Last Price       float64
Open Interest    float64
Strike           float64
Volume           float64
date              object
maturity_date     object
option            object
type              object
dtype: object

In [17]:
time_to_maturity=[]
for i, j in zip(df_dict['AAPL1481846400'].maturity_date,df_dict['AAPL1481846400'].date):

    time_to_maturity.append((i - j).days/365)

In [ ]:
for key in df_dict.keys():
    time_to_maturity = []
    ## grab dataframe, then relevant data
    for i, j in zip(df_dict[key].maturity_date,df_dict[key].date):
        time_to_maturity.append((i - j).days/365)
        
    strike_price = df_dict[key].Strike
    
    strike_price, time_to_maturity = np.meshgrid(strike_price, time_to_maturity)
    
    # generate pseudo-implied volatility by using strike price and time-to-maturity as parameters
    implied_vol = ((strike_price - 150)**2)/(150 * strike_price)/(np.power(time_to_maturity, 0.95))
    
    fig = plot.figure(figsize = (10,5)) ## a plot object
    ax = Axes3D(fig) # create a 3D object/handle
    
    ##plot surface: array row/column stride(step size:2)
    
    surf = ax.plot_surface(strike_price, time_to_maturity, implied_vol, rstride = 2, cstride = 2, cmap = cm.coolwarm, linewidth = 0.5, antialiased = False)
    #set x,y,a labels
    ax.set_xlabel('Strike Price')
    ax.set_ylabel('time to maturity')
    ax.set_zlabel('implied volatility')
    plot.show()
